## Import needed libraries
if you don't have these libraries please run requirements.txt first to install them.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Functions

In [2]:
def make_random_board():
    moves = ['X', 'O', '']
    board = [np.random.choice(moves) for _ in range(9)]
    board = np.array(board)
    board = board.reshape(3, 3)
    return board

In [ ]:
def check_winner(board):
    pass

In [ ]:
def is_valid_board(board):
    pass

In [ ]:
def monte_carlo_simulation(n=5000):
    valid_counter = 0
    for i in range(n):
        board = make_random_board()
        if is_valid_board(board):
            valid_counter += 1
    valid_ratio = valid_counter/n
    return valid_counter, valid_ratio

In [ ]:
def make_all_5_moves():
    pass

In [ ]:
def validate():
    pass

In [ ]:
def plot():
    pass

## Main Menu

In [7]:
if __name__ == '__main__':
    Valid_count, Valid_ratio = monte_carlo_simulation(5000)
    print(f'Number of valid boards: {Valid_count}')
    print(f'Ratio of valid boards: {Valid_ratio.__round__(2)}')

[['' '' 'O']
 ['O' 'O' 'O']
 ['O' 'X' 'O']]


## Estimation & Validation

## Conclusion